In [1]:
# basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# image mainpulation
from PIL import Image

In [13]:
image_path = os.path.join(os.path.expanduser('~'),'Downloads','images') #/Users/ninaadkalla/Downloads/images
images = []

for filename in os.listdir(image_path):
    try:
        if filename.endswith(".jpg"):
            img = Image.open(os.path.join(image_path, filename))
            images.append(img)
    except Exception:
        print(f"❌ The following image file is broken: {filename}")

❌ The following image file is broken: 57446.jpg
❌ The following image file is broken: 58039.jpg
❌ The following image file is broken: 57442.jpg
❌ The following image file is broken: 100253.jpg
❌ The following image file is broken: 57431.jpg
❌ The following image file is broken: 118341.jpg
❌ The following image file is broken: 4042.jpg
❌ The following image file is broken: 58046.jpg
❌ The following image file is broken: 100258.jpg
❌ The following image file is broken: 100249.jpg
❌ The following image file is broken: 58042.jpg
❌ The following image file is broken: 100262.jpg
❌ The following image file is broken: 118316.jpg
❌ The following image file is broken: 57448.jpg
❌ The following image file is broken: 4019.jpg
❌ The following image file is broken: 58037.jpg


In [14]:
image_dimensions = (224, 224)

resized_images = []
for img in images:
    resized_img = img.resize(image_dimensions)
    resized_images.append(resized_img)

tempfile.tif: Old-style JPEG compression support is not configured.
tempfile.tif: Old-style JPEG compression support is not configured.


OSError: -2